###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c) 2019 Daniel Koehn, based on (c)2018 L.A. Barba, G.F. Forsyth [CFD Python](https://github.com/barbagroup/CFDPython#cfd-python), (c)2014 L.A. Barba, I. Hawke, B. Knaepen [Practical Numerical Methods with Python](https://github.com/numerical-mooc/numerical-mooc#practical-numerical-methods-with-python), also under CC-BY.

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# 2D Heat Conduction Example: Thermal Structure of the Lithosphere

After optimizing the runtime of the 2D heat conduction code and validating it against an analytical solution, we will focus in this exercise on the modelling of the thermal structure of the lithosphere.

## The thermal structure of the lithosphere

The problem considered is that of the thermal structure of a lithosphere of $100\; km$ thickness, with an initial linear thermal gradient. Suddenly a plume with $T=1500^oC$ impings at the bottom of the lithosphere. What happens with the thermal structure of the lithosphere? A related (structural geology) problem is that of the cooling of batholites (like the ones in the [Sierra Nevada](https://en.wikipedia.org/wiki/Sierra_Nevada_Batholith))

##### Exercise 1

Solve the above problem using the JIT-optimized FTCS FD-code from the last class.

Let's start by setting up our Python compute environment ...

In [ ]:
# Import libraries
import numpy
from matplotlib import pyplot
%matplotlib inline

# import JIT from Numba
from numba import jit

In [ ]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

Modify the `ftcs_JIT` code below, in order to employ zero flux (Neumann) boundary conditions $\frac{\partial T}{\partial x} = 0$ on the left and right side of the domain

In [ ]:
# FTCS code to solve the 2D heat equation with JIT optimization
# -------------------------------------------------------------
@jit(nopython=True) # use Just-In-Time (JIT) Compilation for C-performance
def ftcs_JIT(T0, nt, dt, dx, dy, alpha):
    """
    Computes and returns the temperature distribution
    after a given number of time steps.
    Explicit integration using forward differencing
    in time and central differencing in space, with
    Neumann conditions (zero-gradient) on top and right
    boundaries and Dirichlet conditions on bottom and
    left boundaries.
    
    Parameters
    ----------
    T0 : numpy.ndarray
        The initial temperature distribution as a 2D array of floats.
    nt : integer
        Maximum number of time steps to compute.
    dt : float
        Time-step size.
    dx : float
        Grid spacing in the x direction.
    dy : float
        Grid spacing in the y direction.
    alpha : float
        Thermal diffusivity.
    
    Returns
    -------
    T : numpy.ndarray
        The temperature distribution as a 2D array of floats.
    """
    # Define some constants.
    sigma_x = alpha * dt / dx**2
    sigma_y = alpha * dt / dy**2
    
    # Integrate in time.
    T = T0.copy()
    
    # Estimate number of grid points in x- and y-direction
    ny, nx = T.shape
    
    # Indices of the model center
    I, J = int(nx / 2), int(ny / 2)  
    
    # Time loop
    for n in range(nt):
        
        # store old temperature field
        Tn = T.copy()
        
        # loop over spatial grid 
        for i in range(1,nx-1):
            for j in range(1,ny-1):
                
                T[j, i] = (Tn[j, i] +
                         sigma_x * (Tn[j, i+1] - 2.0 * Tn[j, i] + Tn[j, i-1]) +
                         sigma_y * (Tn[j+1, i] - 2.0 * Tn[j, i] + Tn[j-1, i]))
        
        # APPLY NEUMANN BOUNDARY CONDITIONS HERE!
          # right boundary
          # left boundary
        
    return T

Let's assume that the width of the lithosphere is $150\; km$ in x-direction and it has a thickness of $100\; km$. The model is discretized with $nx=101$ gridpoints in x-direction and $ny=51$ gridpoints in y-direction. Regarding the thermal properties, assume a thermal diffusivity of the lithosphere $\alpha=\text{1e-6}\; m^2/s$, a temperature of $\text{Tbot} = 1300^oC$ at the bottom of the lithosphere and a temperature $\text{Tsurf} = 0^oC$ at the top. Furthermore, the thermal plume has a width of $\text{Wplume} = 25\;km$ impinging the bottom of the lithosphere at $x=75\; km$. The plume has a temperature of $\text{Tplume} = 1500\; ^oC$. Assume a linear temperature gradient between $\text{Tsurf}$ and $\text{Tbot}$ as initial temperature profile in the lithosphere.

In [ ]:
# Definition of modelling parameters
# ----------------------------------
Lx =   # width of the lithosphere in the x direction [m]
Ly =   # thickness of the lithosphere in the y direction [m]
nx =   # number of points in the x direction
ny =   # number of points in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction
dy = Ly / (ny - 1)  # grid spacing in the y direction
alpha =  # thermal diffusivity of the lithosphere [m^2/s]

# Define the locations along a gridline.
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# DEFINE THE INITIAL TEMPERATURE DISTRIBUTION HERE!
Tbot = 
Tsurf = 
Tplume = 
Wplume =

X, Y = numpy.meshgrid(x,y) # coordinates X,Y required to define T0

# DEFINE INITIAL LINEAR TEMPERATURE GRADIENT HERE!
T0 =

# MODIFY BOTTOM BOUNDARY CONDITION TO INCORPORATE THE PLUME HERE!


We don't want our solution blowing up, so let's find a time step with $\frac{\alpha \Delta t}{\Delta x^2} = \frac{\alpha \Delta t}{\Delta y^2} = \frac{1}{4}$. We also want to model the temperature field for `nt=500` timesteps

In [ ]:
# Set the time-step size based on CFL limit.
sigma = 0.25
dt = sigma * min(dx, dy)**2 / alpha  # time-step size
nt = 500  # number of time steps to compute
print("Maximum modelling time = ",nt*dt/(60*60*24*365*1e6)," million years")

After setting all modelling parameters, we can run the `ftcs_JIT` modelling code to compute the numerical solution after `nt` time steps 

In [ ]:
# Compute the temperature distribution after nt timesteps
T = ftcs_JIT(T0, nt, dt, dx, dy, alpha)

Let's take a look at the resulting temperature field

In [ ]:
# Plot the filled contour of the temperature distribution after 9 Ma 
pyplot.figure(figsize=(11.0, 6.0))
pyplot.xlabel('x [km]')
pyplot.ylabel('y [km]')
levels = numpy.linspace(0., 1500., num=40)
contf = pyplot.contourf(x/1000, y/1000, T, levels=levels)
cbar = pyplot.colorbar(contf)
cbar.set_label('Temperature [°C]')
pyplot.gca().invert_yaxis()

## What we learned:

* How to model the 2D thermal structure of the lithosphere with a plume impinging at the lithosphere/asthenosphere boundary